In [134]:
import otter
grader = otter.Notebook("proj3.ipynb")

In [135]:
import numpy as np
import pandas as pd

In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
!psql postgresql://jovyan@127.0.0.1:5432/ucb_buildings -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'DROP DATABASE IF EXISTS ucb_buildings'
!psql postgresql://jovyan@127.0.0.1:5432/postgres -c 'CREATE DATABASE ucb_buildings'
!psql postgresql://jovyan@127.0.0.1:5432/ucb_buildings -f data/buildings.sql -q

 pg_terminate_backend 
----------------------
 t
(1 row)

DROP DATABASE
CREATE DATABASE
 set_config 
------------
 
(1 row)



In [138]:
from data101_utils import GradingUtil
grading_util = GradingUtil("proj3")
grading_util.prepare_autograder("ucb_buildings")

Successfully opened database connection


In [139]:
grading_util.run_sql("SELECT * FROM data LIMIT 10;")

,time,id,value
0,2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.74
1,2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.51
2,2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.27
3,2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.02
4,2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.77
5,2018-06-12 21:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.54
6,2018-06-12 21:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.30
7,2018-06-12 21:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.04
8,2018-06-12 21:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.80
9,2018-06-12 20:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.57


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


From looking at the schema itself  I see there's some tables that describe the metadata of something or the mapping of two objects; this optimizes both querying operations and storage, referecing the schema now: real_estate_metadata and metadata will cause for an easy lookup of attributes causing less repetition of the same values in multiple tables, therefore optimizing storage.  And mapping describe existing relationships like buildings_site_mapping and mapping; which it's easy for the user to see how the data interacts with each other which will result in the optimization of lookups. 

In [ ]:
--DROP VIEW IF EXISTS view_one;
WITH view_one AS ( 
SELECT DISTINCT buildings_site_mapping.building
FROM buildings_site_mapping
    )

SELECT view_one.building, JSON_AGG(real_estate_metadata)
FROM real_estate_metadata
JOIN view_one ON view_one.building = real_estate_metadata.building_name
GROUP BY view_one.building
HAVING COUNT(real_estate_metadata.building_name) > 1
ORDER BY view_one.building ASC

In [142]:
--result_1b
SELECT *
FROM real_estate_metadata
WHERE building_name = 'ALUMNI HOUSE';


,location,building,building_name,address,city_name,county,category,osfg,bldg_asf,levels,year,owner,mp_code,book_value
0,BERKELEY,1215,ALUMNI HOUSE,CORE C22A2M0PUS,BERKELEY,ALAMEDA,GENERAL,15590,8719,2,1954,UC,P,106819.0
1,IRVINE,9207,ALUMNI HOUSE,CORE CAMPUS,IRVINE,ORANGE,GENERAL,4027,2549,1,1984,UC,P,41981.0
2,FRANCISC SOAN,2032,ALUMNI HOUSE,PARNASS US AVE,FRANCISC OSAN,FRANCISC OSAN,HEALTH SCIENCE,7217,5079,3,1915,UP,P,135923.0


In [143]:
result_1b = grading_util.run_file("1b")
result_1b

,building,json_agg
0,ALUMNI HOUSE,"[{'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}, {'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}]"
1,CAMPBELL,"[{'location': 'BERKELEY', 'building': '1027', 'building_name': 'CAMPBELL', 'address': 'CORE C28A5M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 83373, 'bldg_asf': 50745, 'levels': 7, 'year': '2014', 'owner': 'UC', 'mp_code': 'P', 'book_value': 75256344.0}, {'location': 'LOS ANGELES', 'building': '4294', 'building_name': 'CAMPBELL', 'address': 'PORTOLA P11L8A1Z1A', 'city_name': 'LOS ANGELES', 'county': 'LOS ANGELES', 'category': 'GENERAL', 'osfg': 55401, 'bldg_asf': 32619, 'levels': 5, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 6247192.0}]"
2,HERTZ,"[{'location': 'BERKELEY', 'building': '1423', 'building_name': 'HERTZ', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 31362, 'bldg_asf': 13518, 'levels': 4, 'year': '1958', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3491181.0}, {'location': 'DAVIS', 'building': '9454', 'building_name': 'HERTZ', 'address': '7000 EAST 2A4V0E0 LAKE', 'city_name': 'LIVERMOR E', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 10904, 'bldg_asf': 8183, 'levels': 1, 'year': '1976', 'owner': 'UC', 'mp_code': 'P', 'book_value': 0}]"
3,SOUTH,"[{'location': 'BERKELEY', 'building': '1484', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 30401, 'bldg_asf': 15473, 'levels': 4, 'year': '1873', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1426174.0}, {'location': 'DAVIS', 'building': '3275', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 26594, 'bldg_asf': 15323, 'levels': 4, 'year': '1912', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3585399.0}]"
4,SPROUL,"[{'location': 'BERKELEY', 'building': '1210', 'building_name': 'SPROUL', 'address': 'CORE 2C6A0M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 111197, 'bldg_asf': 66680, 'levels': 6, 'year': '1941', 'owner': 'UC', 'mp_code': 'P', 'book_value': 4278819.0}, {'location': 'DAVIS', 'building': '3815', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 53874, 'bldg_asf': 27913, 'levels': 10, 'year': '1963', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1620880.0}, {'location': 'RIVERSIDE', 'building': 'P5523', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'RIVERSIDE', 'county': 'RIVERSIDE', 'category': 'GENERAL', 'osfg': 80988, 'bldg_asf': 45532, 'levels': 5, 'year': '1965', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1296147.0}]"


In [ ]:

WITH view_one AS  (
SELECT DISTINCT real_estate_metadata.building_name
FROM real_estate_metadata
)


SELECT view_one.building_name, JSON_AGG(buildings_site_mapping)
FROM buildings_site_mapping
JOIN view_one ON view_one.building_name = buildings_site_mapping.building
GROUP BY view_one.building_name
HAVING COUNT(buildings_site_mapping.building) > 1
ORDER BY view_one.building_name ASC

In [145]:
result_1c = grading_util.run_file("1c")
result_1c

,building_name,json_agg
0,EDWARDS FLD,"[{'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}]"
1,FAC CLUB,"[{'site': 'Men's Faculty Club', 'building': 'FAC CLUB'}, {'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}]"
2,HAAS STU BLD,"[{'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}, {'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}]"
3,SIMON,"[{'site': 'Simpson Center', 'building': 'SIMON'}, {'site': 'Boalt and Simon Hall', 'building': 'SIMON'}]"


Things to note: 
1. From the insights in 1b we can't determine a unique building in the sensor data. Sensor data's attributes are value, id and time, while real_estate_metadata tells information about the building itself and none of those attributes have any of the sensor data.
2. It's important to note that  buildings_site_mapping.building couldn' be a foreign key poiting to real_estate_metadata.building_name. As there's multiple repetitions real_estate_metadata.building_name that correspond to a buildings_site_mapping.building entry therefore breaking referential integrity is it has a one-to-many relationship.


### Outlier Detection


In [147]:
--result_2a

WITH median_values AS (
SELECT id, 
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY value) AS median
FROM data
GROUP BY id
), 

two AS ( 
SELECT one.time, one.id, one.value, abs (one.value - view.median) AS premad
FROM data one
JOIN median_values view ON view.id = one.id
),

mad_table AS (
SELECT id,
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY premad) AS mad
FROM two
GROUP BY id
)

 
SELECT 
two.time, 
two.id, 
two.value, 
median_values.median, 
mad_table.mad, 
CASE 
    WHEN ABS(two.value - median_values.median) >  3 * 1.4826 * mad_table.mad
    THEN TRUE 
    WHEN mad_table.mad = 0 THEN FALSE 
    ELSE FALSE 
    END AS is_outlier 

FROM two
JOIN mad_table ON mad_table.id = two.id
JOIN median_values ON median_values.id = two.id
;


,time,id,value,median,mad,is_outlier
0,2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.740,65130.270,22.550,False
1,2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.510,65130.270,22.550,False
2,2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.270,65130.270,22.550,False
3,2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.020,65130.270,22.550,False
4,2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.770,65130.270,22.550,False
...,...,...,...,...,...,...
99995,2018-06-11 02:15:00+00:00,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,5.000,7.129,2.779,False
99996,2018-06-11 02:00:00+00:00,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,3.404,7.129,2.779,False
99997,2018-06-11 01:45:00+00:00,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,4.126,7.129,2.779,False
99998,2018-06-11 01:30:00+00:00,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,6.825,7.129,2.779,False


In [148]:
result_2a = grading_util.run_file("2a")[1]
result_2a

,time,id,value,median,mad,is_outlier
0,2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.500,16.200,True
1,2018-06-07 00:00:00+00:00,a51d3bed-a8b6-5321-96a1-c5542caca70c,75.0,6.186,4.957,True
2,2018-06-07 00:00:00+00:00,a6336afc-cdb7-5f87-a4bf-67411ea1b5cc,54.0,32.000,1.000,True
3,2018-06-07 00:00:00+00:00,a652779f-8433-5ad3-9310-f4f1062e4d04,14.0,8.000,1.000,True
4,2018-06-07 00:00:00+00:00,a760a23a-8de4-5ce3-939a-826bf31f01df,109.0,69.000,7.000,True
...,...,...,...,...,...,...
95,2018-06-07 00:00:00+00:00,a7145dcc-90e5-5bfb-9b1c-277615779c16,4.0,3.000,1.000,False
96,2018-06-07 00:00:00+00:00,a71d821d-d6e6-5ee9-86fa-2b075f13c2cc,99.0,81.000,17.000,False
97,2018-06-07 00:00:00+00:00,a71fbdfd-26ad-562c-b193-9dc414b851f0,122.9,58.700,35.700,False
98,2018-06-07 00:00:00+00:00,a744b292-f4cf-5f85-9e4a-5e626e19d5af,107.0,74.000,18.000,False


<br><br>

---

### Outlier Handling (Winsorization)


In [ ]:
--result_2b

CREATE OR REPLACE VIEW cleaned_data AS

-- BEGIN SOLUTION
WITH median_values AS (
SELECT id, 
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY value) AS median
FROM data
GROUP BY id
), 

two AS ( 
SELECT one.time, one.id, one.value, abs (one.value - view.median) AS premad
FROM data one
JOIN median_values view ON view.id = one.id
),

mad_table AS (
SELECT id,
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY premad) AS mad
FROM two
GROUP BY id
)

 
SELECT 
two.time, 
two.id, 
two.value, 
median_values.median, 
mad_table.mad, 
CASE 
    WHEN mad_table.mad = 0 THEN FALSE
    WHEN ABS(two.value - median_values.median) >  3 * 1.4826 * mad_table.mad
    THEN TRUE 
    ELSE FALSE 
    END AS is_outlier,
    
    CASE
    WHEN mad_table.mad = 0 THEN two.value
    
    WHEN ABS(two.value - median_values.median) <  3 * 1.4826 * mad_table.mad THEN two.value

    WHEN two.value > median_values.median + 3 * 1.4826 * mad_table.mad
    THEN  median_values.median + 3 * 1.4826 * mad_table.mad

   WHEN two.value < median_values.median - 3 * 1.4826 * mad_table.mad
    THEN  median_values.median -  3 * 1.4826 * mad_table.mad
    ELSE two.value 
    END AS clean_value
    
    

FROM two
JOIN mad_table ON mad_table.id = two.id
JOIN median_values ON median_values.id = two.id
;
-- END SOLUTION

(SELECT * FROM cleaned_data WHERE is_outlier ORDER BY time, id LIMIT 50)
UNION ALL
(SELECT * FROM cleaned_data WHERE NOT is_outlier ORDER BY time, id LIMIT 50);


#### Entity Resolution (units + locations)

In [150]:
result_2b = grading_util.run_file("2b")[1]
result_2b

,time,id,value,median,mad,is_outlier,clean_value
0,2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.500,16.200,True,799.554360
1,2018-06-07 00:00:00+00:00,a51d3bed-a8b6-5321-96a1-c5542caca70c,75.0,6.186,4.957,True,28.233745
2,2018-06-07 00:00:00+00:00,a6336afc-cdb7-5f87-a4bf-67411ea1b5cc,54.0,32.000,1.000,True,36.447800
3,2018-06-07 00:00:00+00:00,a652779f-8433-5ad3-9310-f4f1062e4d04,14.0,8.000,1.000,True,12.447800
4,2018-06-07 00:00:00+00:00,a760a23a-8de4-5ce3-939a-826bf31f01df,109.0,69.000,7.000,True,100.134600
...,...,...,...,...,...,...,...
95,2018-06-07 00:00:00+00:00,a7145dcc-90e5-5bfb-9b1c-277615779c16,4.0,3.000,1.000,False,4.000000
96,2018-06-07 00:00:00+00:00,a71d821d-d6e6-5ee9-86fa-2b075f13c2cc,99.0,81.000,17.000,False,99.000000
97,2018-06-07 00:00:00+00:00,a71fbdfd-26ad-562c-b193-9dc414b851f0,122.9,58.700,35.700,False,122.900000
98,2018-06-07 00:00:00+00:00,a744b292-f4cf-5f85-9e4a-5e626e19d5af,107.0,74.000,18.000,False,107.000000


From inspection it seems there are inconsistensies, duplicates and ambiguity; for example in the last rows there's gallons and gpm (gallons per minute) which would create opportunities for cleaning the data as they might refer to the same thing, or mark a clear distinction to in case they are different in order to keep the integrity of the data.

In [154]:
SELECT units FROM metadata;


,units
0,kWh
1,kW
2,kW
3,kW
4,kW
...,...
9504,Gallons
9505,Gpm
9506,Gpm
9507,Gpm


In [ ]:
--result_3b
SELECT COUNT(DISTINCT units) AS num_unique_units, 
COUNT(DISTINCT LOWER(units)) AS num_unique_units_ignore_case

FROM 
metadata


In [155]:
result_3b = grading_util.run_file("3b")
result_3b

,num_unique_units,num_unique_units_ignore_case
0,34,29


In [157]:
grading_util.run_sql("""
SELECT * FROM metadata;
""")

,id,class,site,units
0,fb5267b9-8cf4-5710-979e-5ab7d617e6e5,Main Electric Meter (kWh),2000 Carleton Street,kWh
1,4e7e5872-a3f7-5300-8433-b4dfd0254cb1,Main Electric Meter Demand (kW),2000 Carleton Street,kW
2,a2fbb537-3e9c-5c47-94c9-2b5cc3735f48,Main Electric Meter Min (kW),2000 Carleton Street,kW
3,4eb9d406-9aeb-56ab-aa9d-aaa368679e1f,Main Electric Meter Instantaneous (kW),2000 Carleton Street,kW
4,3c44bdad-d1a9-5155-b786-fb2ce6e1e76e,Main Electric Meter Max (kW),2000 Carleton Street,kW
...,...,...,...,...
9504,13e85c96-6104-53d0-b9d7-935e8f378e5c,Water 1464468 (Gallons),Haviland Hall,Gallons
9505,924e8a05-8534-5b5a-a9a7-1ea4b1c7e14f,Water 1464468 Ave Rate (Gpm),Haviland Hall,Gpm
9506,a6dc58d3-a002-5329-b528-f865228656d0,Water 1464468 Max (Gpm),Haviland Hall,Gpm
9507,20174662-7781-55c4-bd98-4bf1c08c7fac,Water 1464468 Instantaneous (Gpm),Haviland Hall,Gpm




### Checking consistency of units



In [ ]:
--result_3c
WITH view_one AS (
    SELECT class, LOWER(units) as lower_caps
    FROM metadata
    GROUP BY class, lower_caps
)

SELECT 

CASE WHEN COUNT(DISTINCT lower_caps) = 1 THEN TRUE
ELSE FALSE END AS 
are_units_consistent
FROM view_one


In [158]:
result_3c = grading_util.run_file("3c")
result_3c

,are_units_consistent
0,False


Some locations have ambiguous location names that are not specific enough. To provide an example, row 5273 has under location the string 'AG FIELD STAT' and in the address column it actually provide this example. Upon further investigation online, AG FIELD STAT doesn't throw meaning results as it references a government agency.

In [160]:
SELECT * FROM uc_locations;

,loc_id,loc_name
0,1,AG FIELD STAT
1,2,BERKELEY
2,3,DAVIS
3,4,IRVINE
4,5,LOS ANGELES
5,6,MERCED
6,7,RIVERSIDE
7,8,SAN DIEGO
8,9,SAN FRANCISCO
9,10,SANTA BARBARA



---
### Used uc_locations to build a cleaned mapping:



In [ ]:
--result_3e
WITH ranked AS (
SELECT 

real_estate_metadata.building_name AS building_name,

real_estate_metadata.address AS address,

real_estate_metadata.location AS location,

uc_locations.loc_name AS clean_location,

--word_similarity(real_estate_metadata.location, uc_locations.loc_name) AS similarity_score,

RANK() OVER (PARTITION BY real_estate_metadata.location ORDER BY word_similarity(real_estate_metadata.location, uc_locations.loc_name) DESC) AS rank

FROM real_estate_metadata, uc_locations
),

ranked_two AS (
SELECT building_name, address, location, clean_location 
FROM ranked 
WHERE rank = 1

)
SELECT * 
FROM ranked_two;


In [161]:
result_3e = grading_util.run_file("3e")
result_3e

,building_name,address,location,clean_location
0,SC ACAD STR,IRVINE B76L0V1D,AG FIELD STAT,AG FIELD STAT
1,WS 304 HOUSE,OAKLAND 1A7V3E53 W,AG FIELD STAT,AG FIELD STAT
2,H ADMIN HBL DG,UNIVERSIT Y40 R7D0,AG FIELD STAT,AG FIELD STAT
3,D POTTING SDH,HOLTON R10D04 E,AG FIELD STAT,AG FIELD STAT
4,H NUTRI BHARN,UNIVERSIT 4Y0 R7D0,AG FIELD STAT,AG FIELD STAT
...,...,...,...,...
5271,UCDC,RHODE 1IS4L3A5N0 D,SYSTEMWI DE,SYSTEMWIDE
5272,UCPC,MERIDIAN PARKWAY,SYSTEMWI DE,SYSTEMWIDE
5273,RINCON 70,-BLAKE 1H1O1U1SE,SYSTEMWI DE,SYSTEMWIDE
5274,BERKWAY 2120,BERKELEY W70A RYINCON,SYSTEMWI DE,SYSTEMWIDE




## Part 4 – Interpolating Missing Data (gap-filling + linear interpolation)

Goal: Detect missing sensor readings in time, fabricate the missing timestamps, and interpolate reasonable values.


In [ ]:
--result_4a

CREATE OR REPLACE VIEW gaps AS

-- BEGIN SOLUTION
WITH diffs AS (
SELECT 
id, 
DATE_TRUNC('minute', time) AS time,
value, 
LAG(DATE_TRUNC('minute', time), 1) OVER (PARTITION BY id ORDER BY time) AS lag_time,
    
LAG(value, 1) OVER (PARTITION BY id ORDER BY time) AS lag_value, 
    
DATE_TRUNC('minute', time) - LAG(DATE_TRUNC('minute', time), 1) OVER (PARTITION BY id ORDER BY time) AS time_diff
FROM data

)

SELECT * 
FROM diffs
WHERE time_diff >= '30 minutes';
-- END SOLUTION

SELECT * FROM gaps ORDER BY id, time;

In [163]:
result_4a = grading_util.run_file("4a")[1]
result_4a

,id,time,value,lag_time,lag_value,time_diff
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:15:00+00:00,0.056,2018-06-13 08:45:00+00:00,0.056,0 days 00:30:00
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:15:00+00:00,280.800,2018-06-09 03:45:00+00:00,281.600,0 days 00:30:00
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:15:00+00:00,281.460,2018-06-13 08:45:00+00:00,281.390,0 days 00:30:00
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:15:00+00:00,0.004,2018-06-09 03:45:00+00:00,0.006,0 days 00:30:00
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:45:00+00:00,27.273,2018-06-07 00:15:00+00:00,30.000,0 days 00:30:00
...,...,...,...,...,...,...
766,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 08:15:00+00:00,5.400,2018-06-13 07:45:00+00:00,0.012,0 days 00:30:00
767,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 13:00:00+00:00,5.400,2018-06-13 08:15:00+00:00,5.400,0 days 04:45:00
768,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:30:00+00:00,2.700,2018-06-13 13:00:00+00:00,5.400,0 days 01:30:00
769,a8d640b1-c17f-51cf-b06f-de6e704e87c1,2018-06-13 09:15:00+00:00,106.000,2018-06-13 08:45:00+00:00,106.000,0 days 00:30:00


### Filling in timestamps with GENERATE_SERIES

Used GENERATE_SERIES over each gap to manufacture new timestamps every 15 minutes between start and end.




In [ ]:
--result_4b 

CREATE OR REPLACE VIEW complete AS

-- BEGIN SOLUTION

WITH gaps AS (
SELECT 
id, 
DATE_TRUNC('minute', time) AS time,
value, 
LAG(DATE_TRUNC('minute', time), 1) OVER (PARTITION BY id ORDER BY time) AS lag_time,
    
LAG(value, 1) OVER (PARTITION BY id ORDER BY time) AS lag_value, 
    
DATE_TRUNC('minute', time) - LAG(DATE_TRUNC('minute', time), 1) OVER (PARTITION BY id ORDER BY time) AS time_diff
FROM data
),

gaps_two AS (
SELECT * 
FROM gaps
WHERE time_diff >= '30 minutes'
) 

SELECT 
id, time, value

FROM data 

UNION ALL

SELECT 

id, 

generated_time AS time,

NULL AS value 

FROM gaps_two

CROSS JOIN LATERAL GENERATE_SERIES(lag_time + '15 minutes'::INTERVAL, 
time - '15 minutes'::INTERVAL, '15 minutes'::INTERVAL) AS generated_time;
;
-- END SOLUTION

SELECT * FROM complete ORDER BY id, time LIMIT 100;

In [165]:
grading_util.run_sql("""
SELECT *
FROM GENERATE_SERIES(1, 10, 3);
""")

,generate_series
0,1
1,4
2,7
3,10


In [167]:
result_4b = grading_util.run_file("4b")[1]
result_4b

,id,time,value
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:09+00:00,65085.99
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:09+00:00,65086.16
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:09+00:00,65086.35
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:09+00:00,65086.52
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:09+00:00,65086.71
...,...,...,...
95,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 23:45:09+00:00,65100.44
96,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:00:09+00:00,65100.63
97,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:15:09+00:00,65100.80
98,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:30:09+00:00,65100.97


In [169]:
grading_util.run_sql("""
SELECT *
FROM complete;
""")

,id,time,value
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 23:00:09+00:00,65182.74
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:45:09+00:00,65182.51
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:30:09+00:00,65182.27
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:15:10+00:00,65182.02
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:00:09+00:00,65181.77
...,...,...,...
102381,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 13:45:00+00:00,NaN
102382,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:00:00+00:00,NaN
102383,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:15:00+00:00,NaN
102384,a8d640b1-c17f-51cf-b06f-de6e704e87c1,2018-06-13 09:00:00+00:00,NaN


### Question 4c:**Linear interpolation (forward / backward passes):**

In [170]:
--result_4c
CREATE TABLE temp (
    id INTEGER,
    num INTEGER
);

INSERT INTO temp VALUES
    (1, 1),
    (2, 2),
    (3, NULL),
    (4, NULL),
    (5, 3),
    (6, NULL),
    (7, 4);

SELECT COALESCE_AGG(num) OVER (ORDER BY id) AS run_start FROM temp;

DROP TABLE temp;
coalesce_agg_result

,run_start
0,1
1,2
2,2
3,2
4,3
5,3
6,4


#### Forward Pass


In [ ]:
-- result_4ci 

CREATE OR REPLACE VIEW forward AS


    WITH pre_view AS (
    SELECT
    
    id,
    
    time, 

    value,
    
    RANK() OVER (ORDER BY id) AS run
        
    FROM complete_provided
    
    )
        
    SELECT 
    id, 
    
    time, 

    value, 
    
    run,
    
    COALESCE_AGG(value) OVER (ORDER BY run) AS run_start,
    
    LEAD(value, 1) OVER (ORDER BY run) AS next_val 
    
    FROM pre_view;

-- END SOLUTION

SELECT * FROM forward WHERE next_val IS NOT NULL ORDER BY run LIMIT 100;

In [212]:
result_4ci = grading_util.run_file("4ci")[1]
result_4ci

,id,time,value,run,run_start,next_val
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:09+00:00,65090.004847,1,65095.063848,65155.239319
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 11:00:09+00:00,65155.239319,1,65095.063848,65154.187017
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 11:30:09+00:00,65154.187017,1,65095.063848,65157.141714
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 11:45:10+00:00,65157.141714,1,65095.063848,65154.773395
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 12:00:09+00:00,65154.773395,1,65095.063848,65159.105666
...,...,...,...,...,...,...
95,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 10:45:09+00:00,65180.923290,1,65095.063848,65175.487194
96,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 11:00:09+00:00,65175.487194,1,65095.063848,65178.254159
97,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 11:15:09+00:00,65178.254159,1,65095.063848,65173.085489
98,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 11:30:09+00:00,65173.085489,1,65095.063848,65178.994692


#### Backward Pass


In [ ]:
-- result_4cii
CREATE OR REPLACE VIEW backward AS
-- BEGIN SOLUTION
  WITH pre_view AS (
    SELECT
    
    id,
    
    time, 

    value,
    
    RANK() OVER (ORDER BY id) AS run
        
    FROM complete
    
    ),

    forward AS (
    
    SELECT 
    id, 
    
    time, 

    value, 
    
    run,
    
    COALESCE_AGG(value) OVER (ORDER BY run) AS run_start,
    
    LEAD(value, 1) OVER (ORDER BY run) AS next_val 
    
    FROM pre_view
    )

        
    SELECT 
    id, 
    
    time, 

    value, 
    
    run,
    run_start,
     next_val,

    COALESCE_AGG(next_val) OVER (PARTITION BY run ORDER BY id DESC)AS run_end, 

    ROW_NUMBER() OVER (PARTITION BY run ORDER BY id) AS run_rank,

    COUNT(*) OVER (PARTITION BY run) AS run_size

     
    FROM forward;
-- END SOLUTION

SELECT * FROM backward WHERE run_size > 2 ORDER BY id, run, run_rank LIMIT 100;

In [214]:

    SELECT *
    FROM complete;


,id,time,value
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 23:00:09+00:00,65182.74
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:45:09+00:00,65182.51
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:30:09+00:00,65182.27
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:15:10+00:00,65182.02
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:00:09+00:00,65181.77
...,...,...,...
102381,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 13:45:00+00:00,NaN
102382,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:00:00+00:00,NaN
102383,a842919a-dcd1-5d91-b1df-f70c988cabfe,2018-06-13 14:15:00+00:00,NaN
102384,a8d640b1-c17f-51cf-b06f-de6e704e87c1,2018-06-13 09:00:00+00:00,NaN


In [215]:
result_4cii = grading_util.run_file("4cii")[1]
result_4cii

,id,time,value,run,run_start,next_val,run_end,run_rank,run_size
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:45:09+00:00,65182.51,1,65085.99,65182.27,0.052,1,573
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:30:09+00:00,65182.27,1,65085.99,65182.02,0.052,2,573
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:15:10+00:00,65182.02,1,65085.99,65181.77,0.052,3,573
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 22:00:09+00:00,65181.77,1,65085.99,65181.54,0.052,4,573
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-12 21:45:09+00:00,65181.54,1,65085.99,65181.30,0.052,5,573
...,...,...,...,...,...,...,...,...,...
95,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 23:00:09+00:00,65162.09,1,65085.99,65161.86,0.052,96,573
96,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 22:45:09+00:00,65161.86,1,65085.99,65161.61,0.052,97,573
97,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 22:30:09+00:00,65161.61,1,65085.99,65161.36,0.052,98,573
98,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-11 22:15:09+00:00,65161.36,1,65085.99,65161.12,0.052,99,573


#### Interpolate



In [ ]:
-- result_4ciii

CREATE OR REPLACE VIEW likely_data AS
-- BEGIN SOLUTION
 

WITH pre_view AS (
    SELECT 
    id, 
    time,
    value, 
    SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END) 
         OVER (ORDER BY id) AS run,
         
 COALESCE_AGG(value) OVER (ORDER BY id) AS run_start,
 
    CASE WHEN value IS NULL 
        THEN lead(value, 1) OVER (ORDER BY id)
            ELSE NULL
             END AS next_val

             FROM  complete_provided

     ), 

    prev_two AS (

    SELECT 
    id, 
    time,
    value, 
    run, 
    run_start, 
    next_val, 
    
   CASE WHEN id IS NOT NULL THEN value
            ELSE COALESCE_AGG(next_val) OVER
            (PARTITION BY run ORDER BY id DESC)
        END AS run_end, 

    ROW_NUMBER() OVER (PARTITION BY run ORDER BY id) AS run_rank,
    
    COUNT(*) OVER (PARTITION BY run) AS run_size
    

    FROM pre_view
     
     )
    SELECT 
    id, 
    time,
    value, 
    run, 
    run_start, 
    next_val, 
    run_end, 

    run_rank,
    
    run_size,
    
    run_start + (run_rank - 1)*((run_end-run_start)/(run_size - 1))
         AS interpolated
         
    FROM prev_two;  

    
    
-- END SOLUTION

SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

In [217]:
SELECT * 
FROM complete_provided;
    


,id,time,value
0,a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,NaN
1,a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:00:00+00:00,NaN
2,a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:00:00+00:00,NaN
3,a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:00:00+00:00,NaN
4,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,NaN
...,...,...,...
102381,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,2018-06-11 02:15:00+00:00,10.322252
102382,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,2018-06-11 02:00:00+00:00,8.774114
102383,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,2018-06-11 01:45:00+00:00,12.540071
102384,a8dc3e90-6bb7-56e4-bf10-bf07694da31b,2018-06-11 01:30:00+00:00,14.974451


In [218]:
result_4ciii = grading_util.run_file("4ciii")[1]
result_4ciii

,id,time,value,run,run_start,next_val,run_end,run_rank,run_size,interpolated
0,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:09+00:00,65090.004847,573,65095.063848,None,65090.004847,1,573,65095.063848
1,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:09+00:00,65095.063848,573,65095.063848,None,65095.063848,573,573,65095.063848
2,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:09+00:00,65088.888893,573,65095.063848,None,65088.888893,572,573,65088.899688
3,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:09+00:00,65091.749186,573,65095.063848,None,65091.749186,571,573,65091.760775
4,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:09+00:00,65094.524657,573,65095.063848,None,65094.524657,570,573,65094.527485
...,...,...,...,...,...,...,...,...,...,...
95,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 23:45:09+00:00,65107.778700,573,65095.063848,None,65107.778700,479,573,65105.689197
96,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:00:09+00:00,65101.823095,573,65095.063848,None,65101.823095,478,573,65100.700492
97,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:15:09+00:00,65108.302662,573,65095.063848,None,65108.302662,477,573,65106.080763
98,a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-08 00:30:09+00:00,65103.203499,573,65095.063848,None,65103.203499,476,573,65101.823174
